In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
csv_path  = "M2_2.18mM_pressure_o2_release.csv"
time_col  = "Time (s)"                     # your column is actually hours
press_col = "DWT denoised pressure (kPa)"  # pressure in kPa

In [2]:
df = pd.read_csv(csv_path)
df.columns = [c.strip() for c in df.columns]

### Using the full equation to see if there is a difference in the total umol produced by 1. keeping pressure and other units in Kpa   and 2. Keeping pressure and other units in atm




""" Units used:

KH = 1.283e-5          
R = 8.314462  


for atm,
KH = 1.3e-3                        
R=   0.082057366080960            


Sticking with K instead of C. The ideal gas law and henry's law both scale with kelvin so using celcius scaling will become inappropriate.


Using : Vg / (R * T) + kH * Vsol


"""

In [4]:
t_h   = pd.to_numeric(df[time_col], errors="coerce")
P_kPa = pd.to_numeric(df[press_col], errors="coerce")
mask = t_h.notna() & P_kPa.notna()
t_h, P_kPa = t_h[mask].reset_index(drop=True), P_kPa[mask].reset_index(drop=True)
t_h = t_h - t_h.iloc[0]  # start time at 0 h
P_atm = P_kPa / 101.325


# Geometry & constants
Vg   = 0.002     # L
Vsol = 0.006     # L
T_K  = 297.15    # K

# --- (A) kPa-guided calculation ---
R_kPa  = 8.31446261815324              # L·kPa·mol^-1·K^-1
kH_kPa = 1.3e-3 / 101.325              # mol·L^-1·kPa^-1 (~1.283e-5)
A_kPa  = Vg/(R_kPa*T_K) + kH_kPa*Vsol  # mol/kPa
n_tot_kPa_umol = P_kPa * A_kPa * 1e6   # µmol

# --- (B) atm-guided calculation ---
R_atm  = 0.082057366080960             # L·atm·mol^-1·K^-1
kH_atm = 1.3e-3                        # mol·L^-1·atm^-1
A_atm  = Vg/(R_atm*T_K) + kH_atm*Vsol  # mol/atm
n_tot_atm_umol = P_atm * A_atm * 1e6   # µmol

# Residuals
final_kpa = max(n_tot_kPa_umol)
final_atm = max(n_tot_atm_umol)
diff_umol = final_kpa - final_atm
print(f"max values of o2 in umol produced by keeping pressure in kpa : {final_kpa}")
print(f"max values of o2 in umol produced by keeping pressure in atm : {final_atm}")
print(f"Max |difference|: {np.abs(diff_umol):.6e} µmol")

max values of o2 in umol produced by keeping pressure in kpa : 5.325480890636497
max values of o2 in umol produced by keeping pressure in atm : 5.325480890636478
Max |difference|: 1.865175e-14 µmol


The above results because of Kpa vs atm just looks like rounding noise. There is no major difference in the total umol produced by both

### Vant hoff modelling for KH


### KK modelling for KH
